<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/Fundamentus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
"""
Scraper FundamentuS
- Rodar para lista de tickers (ex.: ["PETR4"])
- Retorna/mostra um DataFrame pandas (uma linha por ticker)
- grava em Google Sheets
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from IPython.display import display
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

In [42]:
# B3 STOCKS FUNDAMENTUS ROUTINE
#
tickers = ['^BVSP','ABEV3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CMIN3.SA','CPFE3.SA','CPLE6.SA','ELET6.SA','EMBR3.SA','ITUB4.SA','IVVB11.SA','JHSF3.SA','KLBN11.SA','MBRF3.SA','LEVE3.SA','LREN3.SA','PETR4.SA','POMO4.SA','RECV3.SA','SBSP3.SA','TGMA3.SA','VALE3.SA','VIVA3.SA','VIVT3.SA','WEGE3.SA']
#

In [43]:
# Remove ".SA" from stock dataframe tickers
tickersadjusted = [nome.replace('.SA', '') for nome in tickers]
# Remove "^BVSP" ticker
if "^BVSP" in tickersadjusted:
    tickersadjusted.remove("^BVSP")

In [44]:
# tickers = ["PETR4","VALE3"]
tickers = tickersadjusted
print (tickers)

['ABEV3', 'BBAS3', 'BBDC4', 'BBSE3', 'CMIN3', 'CPFE3', 'CPLE6', 'ELET6', 'EMBR3', 'ITUB4', 'IVVB11', 'JHSF3', 'KLBN11', 'MBRF3', 'LEVE3', 'LREN3', 'PETR4', 'POMO4', 'RECV3', 'SBSP3', 'TGMA3', 'VALE3', 'VIVA3', 'VIVT3', 'WEGE3']


In [45]:
# ---------------- utilitários ----------------

'''def parse_brazil_number(s):
    """Converte textos numéricos BR para float ou retorna string.
       Também lida com percentuais (retorna float representando o número, ex: '41,7%' -> 41.7)."""
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    st = str(s).strip()
    if st == "" or st == "-" or st.lower() in ("n/a","na"):
        return None
    st = st.replace("\xa0", " ").strip()
    # detectar percentual
    is_pct = False
    if "%" in st:
        is_pct = True
        st = st.replace("%", "")
    # limpar "R$" e espaços
    st = st.replace("R$", "").strip()
    # se contém letras longas (ex: "PN") -> não é número
    if re.search(r'[A-Za-z]{2,}', st) and not re.search(r'\d', st):
        return st
    # formatos:
    try:
        if "." in st and "," in st:
            num = float(st.replace(".", "").replace(",", "."))
        elif "," in st and "." not in st:
            num = float(st.replace(",", "."))
        elif "." in st and "," not in st:
            # likely thousand sep -> remove dots
            num = float(st.replace(".", ""))
        else:
            num = float(st)
        return float(num) if not is_pct else float(num)
    except:
        return st

def extract_label_value_pairs_from_tr(tr):
    """Emparelha células adjacentes num <tr> retornando lista de (label, value)."""
    cells = [c.get_text(" ", strip=True) for c in tr.find_all(['td','th'])]
    pairs = []
    i = 0
    n = len(cells)
    while i < n:
        label = cells[i].strip()
        if label == "":
            i += 1
            continue
        j = i + 1
        while j < n and cells[j].strip() == "":
            j += 1
        if j >= n:
            break
        value = cells[j].strip()
        label_norm = re.sub(r'[:\s]+$', '', label)
        pairs.append((label_norm, value))
        i = j + 1
    return pairs

# ---------------- mapping de labels ----------------
# ordem importante: padrões específicos (margem ebit) antes de padrões genéricos como 'ebit'
LABEL_MAP = [
    (r'^(papel|ticker)$', 'ticker'),
    (r'cotac', 'cotacao'),
    (r'data.*ult', 'data_ultima_cotacao'),
    (r'min 52', 'min_52_sem'),
    (r'max 52', 'max_52_sem'),
    (r'vol .*2m|vol .*med', 'vol_med_2m'),
    (r'^setor$', 'setor'),
    (r'^subsetor$', 'subsetor'),
    (r'valor de mercado|valor mercado', 'valor_mercado'),
    (r'valor da firma|valor firma', 'valor_firma'),
    (r'n(ro|º)|n(ro|º).*a[cç]oes|número.*a[cç]oes|nro.*a[cç]oes', 'nro_acoes'),
    (r'p\/l|p\.?\/l', 'pl'),
    (r'lpa', 'lpa'),
    (r'p\/vp|p\.?\/vp', 'p_vp'),
    (r'vpa', 'vpa'),
    (r'marg.*ebit', 'marg_ebit'),            # específico: Marg. EBIT (percentual)
    (r'p\/ebit|p\.?\/ebit', 'p_ebit'),
    (r'marg.*brut|margem.*bruta', 'marg_bruta'),
    (r'psr', 'psr'),
    (r'^ebit$', 'ebit'),                     # específico: EBIT valor monetário
    (r'p_?ativos|p\/ativos', 'p_ativos'),
    (r'marg.*liquida|margem.*liquida', 'marg_liquida'),
    (r'p[_\s]?cap[_\s]?giro|p cap giro', 'p_cap_giro'),
    (r'p[_\s]?ativ[_\s]?circ[_\s]?liq', 'p_ativ_circ_liq'),
    (r'roic', 'roic'),
    (r'div.*yield|dividend.*yield', 'dividend_yield'),
    (r'roe', 'roe'),
    (r'ev.*ebitda|ev / ebitda', 'ev_ebitda'),
    (r'liquidez corr|liquidez_corr|liquidez', 'liquidez_corr'),
    (r'ev.*ebit|ev / ebit', 'ev_ebit'),
    (r'div br.*patrim|div br patrim', 'div_br_patrim'),
    (r'cres.*rec|cres_rec_5a', 'cres_rec_5a'),
    (r'giro.*ativo|giro_ativos', 'giro_ativos'),
    (r'^ativo$', 'ativo'),
    (r'disponibilidades', 'disponibilidades'),
    (r'ativo circulante|ativo_circulante', 'ativo_circulante'),
    (r'div.*bruta', 'div_bruta'),
    (r'div.*l[ií]quida', 'div_liquida'),
    (r'patrim(o|ô)nio', 'patrimonio_liquido'),
    (r'receita liquida', 'receita_liquida_12m'),
    (r'lucro l[ií]quido', 'lucro_liquido_12m'),
    (r'oscila', 'oscilacoes'),
    (r'empresa', 'empresa'),
    (r'tipo', 'tipo'),
    (r'subsetor', 'subsetor'),
    (r'ultimos 12 meses|ultimos_12_meses', 'ultimos_12_meses'),
]

def normalize_label(label):
    lab = label.lower().strip()
    trans = str.maketrans("áàãâéêíóôõúüç","aaaaeeiooouuc")
    lab_no = lab.translate(trans)
    for pat, std in LABEL_MAP:
        if re.search(pat, lab_no):
            return std
    s = re.sub(r'[:\.\-\/\(\)]', ' ', lab_no)
    s = re.sub(r'[^0-9a-z\s]', '', s)
    s = re.sub(r'\s+', '_', s).strip('_')
    return s if s else label.lower()

# ---------------- parse da página ----------------

def parse_fundamentus_page(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    raw = {}
    # coletar pares originais
    for table in soup.find_all('table'):
        for tr in table.find_all('tr'):
            pairs = extract_label_value_pairs_from_tr(tr)
            for label, value in pairs:
                if not label:
                    continue
                # priorizar primeiro não vazio
                if label in raw:
                    if raw[label] in (None, "", "-") and value:
                        raw[label] = value
                else:
                    raw[label] = value

    # garantir captura explícita de 'Setor' se existir (usa label exato)
    # fazer busca em raw por chaves que são exatamente 'Setor' (caso-insens)
    sector_val = None
    for k in list(raw.keys()):
        if k.strip().lower() == 'setor':
            sector_val = raw[k]
            break
    # normalização
    normalized = {}
    for k, v in raw.items():
        std = normalize_label(k)
        parsed = parse_brazil_number(v) if std not in ('page_title','headline') else v
        normalized[std] = parsed

    # se 'setor' foi detectado explicitamente em raw, sobrescrever a normalização com seu valor
    if sector_val is not None:
        normalized['setor'] = parse_brazil_number(sector_val)

    # tratar Marg. EBIT e EBIT especificamente se apareceram como textos diferentes
    # procurar em raw label que contenha 'marg' e 'ebit' (caso exista)
    for k in raw.keys():
        kl = k.lower()
        if 'marg' in kl and 'ebit' in kl:
            normalized['marg_ebit'] = parse_brazil_number(raw[k])
        if kl.strip() == 'ebit' or re.fullmatch(r'ebit', kl, flags=re.I):
            normalized['ebit'] = parse_brazil_number(raw[k])

    return normalized

# ---------------- montar DataFrame e pós-processar ----------------

def get_many_tickers_fundamentus_df(tickers, pause=0.6):
    session = requests.Session()
    session.headers.update(HEADERS)
    rows = []
    cols_union = set()

    for tic in tickers:
        url = f"https://www.fundamentus.com.br/detalhes.php?papel={tic.upper()}"
        r = session.get(url, timeout=15)
        r.raise_for_status()
        info = parse_fundamentus_page(r.text)
        info['ticker'] = tic.upper()
        rows.append(info)
        cols_union.update(info.keys())
        sleep(pause)

    cols = ['ticker'] + sorted([c for c in cols_union if c != 'ticker'])
    df = pd.DataFrame(rows, columns=cols)

    # ----- excluir colunas solicitadas (inclui page_title) -----
    to_drop_tokens = {'empresa','oscilacoes','page_title','pagetitle','papel','dia','mes','30_dias','30dias','tipo','subsetor','ultimos_12_meses'}
    drop_cols = []
    for c in df.columns:
        cl = c.lower()
        for t in to_drop_tokens:
            if t in cl:
                drop_cols.append(c)
                break
    # remover colunas de anos >= 2020
    for year in range(2020, 2100):
        y = str(year)
        for c in df.columns:
            if y in c:
                drop_cols.append(c)
    # remover colunas relacionadas a "últimos 3 meses" se existirem (tokens comuns)
    for c in df.columns:
        if re.search(r'3\s*mes|ult.*3', c, flags=re.I):
            drop_cols.append(c)
    drop_cols = sorted(set(drop_cols))
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

        # ----- Conversão para milhões (mantendo decimais em indicadores) -----
    indicators = {
        'pl','p_vp','p_ebit','psr','dividend_yield','roe','roic','lpa','vpa',
        'p_ativos','p_cap_giro','p_ativ_circ_liq','ev_ebitda','ev_ebit',
        'liquidez_corr','div_br_patrim','cres_rec_5a','giro_ativos'
    }

    for col in df.columns:
        # tratar nro_acoes: inteiro completo
        if col == 'nro_acoes':
            df[col] = pd.to_numeric(df[col], errors='coerce').apply(lambda x: int(x) if pd.notna(x) else x)
            df[col] = df[col].astype(object)
            continue

        ser_num = pd.to_numeric(df[col], errors='coerce')
        if ser_num.notna().any():
            if col in indicators:
                # manter decimais; especial: p_ativ_circ_liq com 2 casas decimais
                if col == 'p_ativ_circ_liq':
                    df[col] = ser_num.apply(lambda x: round(float(x), 2) if pd.notna(x) else x)
                else:
                    df[col] = ser_num.apply(lambda x: float(x) if pd.notna(x) else x)
            elif col == 'ebit':
                # EBIT em milhões, inteiro, sem decimais e sem notação científica
                df[col] = ser_num.apply(
                    lambda x: int(round(x / 1_000_000)) if pd.notna(x) and abs(x) >= 1_000 else (
                        int(x) if pd.notna(x) else x
                    )
                )
                df[col] = df[col].astype(object)
            else:
                # converter grandes valores para milhões (arredondar)
                def conv(v):
                    if pd.isna(v):
                        return v
                    try:
                        if abs(v) >= 1_000_000:
                            return int(round(v / 1_000_000))  # convertido para milhões, sem decimais
                        if float(v).is_integer():
                            return int(v)
                        return float(v)
                    except:
                        return v
                df[col] = ser_num.apply(conv)
                df[col] = df[col].astype(object)
        else:
            df[col] = df[col].astype(object)

    # ----- Garantir setor correto: se 'setor' vazio e 'subsetor' existir, usar subsetor as fallback -----
    if 'setor' not in df.columns and 'subsetor' in df.columns:
        df['setor'] = df['subsetor']
    # prefer explicit 'setor' value if present; we already mapped explicit earlier

    # ----- Reordenar colunas (best-effort) -----
    desired_order = [
        "ticker","cotacao","data_ultima_cotacao","min_52_sem","max_52_sem","vol_med_2m",
        "setor","valor_mercado","valor_firma","nro_acoes","pl","lpa","p_vp","vpa","p_ebit",
        "marg_bruta","psr","marg_ebit","p_ativos","marg_liquida","p_cap_giro","p_ativ_circ_liq",
        "roic","dividend_yield","roe","ev_ebitda","liquidez_corr","ev_ebit","div_br_patrim",
        "cres_rec_5a","giro_ativos","ativo","disponibilidades","ativo_circulante","div_bruta",
        "div_liquida","patrimonio_liquido","receita_liquida_12m","ebit","lucro_liquido_12m"
    ]

    final_cols = []
    existing = list(df.columns)
    used = set()
    for want in desired_order:
        match = None
        if want in existing:
            match = want
        else:
            # substring match
            for c in existing:
                if c in used:
                    continue
                cl = c.lower()
                if want.replace('_',' ') in cl or all(tok in cl for tok in want.split('_') if tok):
                    match = c
                    break
        if match:
            final_cols.append(match)
            used.add(match)
    # append remaining columns
    for c in existing:
        if c not in used:
            final_cols.append(c)
    df = df.reindex(columns=final_cols)

    # ----- Exibição: formatar colunas de indicadores com decimais adequados -----
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 400)
    # formatting: keep floats showing meaningful decimals
    def fmt(x):
        if pd.isna(x):
            return ""
        if isinstance(x, int):
            return f"{x:,}"
        if isinstance(x, float):
            # itens que are indicators keep up to 2 decimals except when integer-like
            return f"{x:,.2f}".rstrip('0').rstrip('.')
        return str(x)
    # We will not convert df values to strings globally; display will show numeric types.
    # But ensure p_ativ_circ_liq is float with 2 decimals (already rounded above)

    return df

# ---------------- Execução exemplo ----------------

if __name__ == "__main__":
    # tickers = ["PETR4","VALE3"]
    df = get_many_tickers_fundamentus_df(tickers, pause=0.6)
    display(df)
    # print("\nDataFrame (uma linha por ticker):")
    # print(df.to_string(index=False))'''


<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1226580944.py:22: SyntaxWarning: invalid escape sequence '\d'
  if re.search(r'[A-Za-z]{2,}', st) and not re.search(r'\d', st):


'def parse_brazil_number(s):\n    """Converte textos numéricos BR para float ou retorna string.\n       Também lida com percentuais (retorna float representando o número, ex: \'41,7%\' -> 41.7)."""\n    if s is None:\n        return None\n    if isinstance(s, (int, float)):\n        return float(s)\n    st = str(s).strip()\n    if st == "" or st == "-" or st.lower() in ("n/a","na"):\n        return None\n    st = st.replace("\xa0", " ").strip()\n    # detectar percentual\n    is_pct = False\n    if "%" in st:\n        is_pct = True\n        st = st.replace("%", "")\n    # limpar "R$" e espaços\n    st = st.replace("R$", "").strip()\n    # se contém letras longas (ex: "PN") -> não é número\n    if re.search(r\'[A-Za-z]{2,}\', st) and not re.search(r\'\\d\', st):\n        return st\n    # formatos:\n    try:\n        if "." in st and "," in st:\n            num = float(st.replace(".", "").replace(",", "."))\n        elif "," in st and "." not in st:\n            num = float(st.replace(",

In [46]:
"""
Scraper Fundamentus — versão final pedido
- Lista de tickers fornecida
- DataFrame final nomeado como `fundamentus`
- Mostra resultado via display() e print()
Requisitos: pip install requests beautifulsoup4 pandas
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from IPython.display import display

HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# ---------- utilitários ----------
def parse_brazil_number(s):
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    st = str(s).strip()
    if st in ("", "-", "n/a", "na"):
        return None
    st = st.replace("\xa0", " ").strip()
    if "%" in st:
        st = st.replace("%", "")
    st = st.replace("R$", "").strip()
    # if it's text like 'PN' or 'ON' return as-is
    if re.fullmatch(r'[A-Za-z\.\s]+', st) and not re.search(r'\d', st):
        return st
    try:
        if "." in st and "," in st:
            return float(st.replace(".", "").replace(",", "."))
        if "," in st and "." not in st:
            return float(st.replace(",", "."))
        if "." in st and "," not in st:
            return float(st.replace(".", ""))
        return float(st)
    except:
        return st

def extract_label_value_pairs_from_tr(tr):
    cells = [c.get_text(" ", strip=True) for c in tr.find_all(['td','th'])]
    pairs = []
    i = 0
    n = len(cells)
    while i < n:
        label = cells[i].strip()
        if label == "":
            i += 1
            continue
        j = i + 1
        while j < n and cells[j].strip() == "":
            j += 1
        if j >= n:
            break
        value = cells[j].strip()
        label_norm = re.sub(r'[:\s]+$', '', label)
        pairs.append((label_norm, value))
        i = j + 1
    return pairs

# ---------- mapping de labels (mantido do código que funcionou) ----------
LABEL_MAP = [
    (r'^(papel|ticker)$', 'ticker'),
    (r'cotac', 'cotacao'),
    (r'data.*ult', 'data_ultima_cotacao'),
    (r'min 52', 'min_52_sem'),
    (r'max 52', 'max_52_sem'),
    (r'vol .*2m|vol .*med', 'vol_med_2m'),
    (r'^setor$', 'setor'),
    (r'^subsetor$', 'subsetor'),
    (r'valor de mercado|valor mercado', 'valor_mercado'),
    (r'valor da firma|valor firma', 'valor_firma'),
    (r'n(ro|º)|n(ro|º).*a[cç]oes|número.*a[cç]oes|nro.*a[cç]oes', 'nro_acoes'),
    (r'p\/l|p\.?\/l', 'pl'),
    (r'lpa', 'lpa'),
    (r'p\/vp|p\.?\/vp', 'p_vp'),
    (r'vpa', 'vpa'),
    (r'marg.*ebit', 'marg_ebit'),
    (r'p\/ebit|p\.?\/ebit', 'p_ebit'),
    (r'marg.*brut|margem.*bruta', 'marg_bruta'),
    (r'psr', 'psr'),
    (r'^ebit$', 'ebit'),
    (r'p_?ativos|p\/ativos', 'p_ativos'),
    (r'marg.*liquida|margem.*liquida', 'marg_liquida'),
    (r'p[_\s]?cap[_\s]?giro|p cap giro', 'p_cap_giro'),
    (r'p[_\s]?ativ[_\s]?circ[_\s]?liq', 'p_ativ_circ_liq'),
    (r'roic', 'roic'),
    (r'div.*yield|dividend.*yield', 'dividend_yield'),
    (r'roe', 'roe'),
    (r'ev.*ebitda|ev / ebitda', 'ev_ebitda'),
    (r'liquidez corr|liquidez_corr|liquidez', 'liquidez_corr'),
    (r'ev.*ebit|ev / ebit', 'ev_ebit'),
    (r'div br.*patrim|div br patrim', 'div_br_patrim'),
    (r'cres.*rec|cres_rec_5a', 'cres_rec_5a'),
    (r'giro.*ativo|giro_ativos', 'giro_ativos'),
    (r'^ativo$', 'ativo'),
    (r'disponibilidades', 'disponibilidades'),
    (r'ativo circulante|ativo_circulante', 'ativo_circulante'),
    (r'div.*bruta', 'div_bruta'),
    (r'div.*l[ií]quida', 'div_liquida'),
    (r'patrim(o|ô)nio', 'patrimonio_liquido'),
    (r'receita liquida', 'receita_liquida_12m'),
    (r'lucro l[ií]quido', 'lucro_liquido_12m'),
    (r'empresa', 'empresa'),
    (r'oscila', 'oscilacoes'),
    (r'tipo', 'tipo')
]

def normalize_label(label):
    lab = label.lower().strip()
    trans = str.maketrans("áàãâéêíóôõúüç","aaaaeeiooouuc")
    lab_no = lab.translate(trans)
    for pat, std in LABEL_MAP:
        if re.search(pat, lab_no):
            return std
    s = re.sub(r'[:\.\-\/\(\)]', ' ', lab_no)
    s = re.sub(r'[^0-9a-z\s]', '', s)
    s = re.sub(r'\s+', '_', s).strip('_')
    return s if s else label.lower()

# ---------- parse da página ----------
def parse_fundamentus_page(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    raw = {}
    for table in soup.find_all('table'):
        for tr in table.find_all('tr'):
            pairs = extract_label_value_pairs_from_tr(tr)
            for label, value in pairs:
                if not label:
                    continue
                # priorizar primeiro não vazio
                if label in raw and raw[label] in (None, "", "-") and value:
                    raw[label] = value
                else:
                    raw[label] = value

    normalized = {}
    for k, v in raw.items():
        std = normalize_label(k)
        parsed = parse_brazil_number(v)
        normalized[std] = parsed

    return normalized

# ---------- função principal ----------
def get_many_tickers_fundamentus_df(tickers, pause=0.6):
    session = requests.Session()
    session.headers.update(HEADERS)
    rows = []
    cols_union = set()

    for orig_tic in tickers:
        # normalizar ticker para consulta: remover '.SA' e leading '^'
        query_tic = orig_tic.split('.')[0].lstrip('^').upper()
        url = f"https://www.fundamentus.com.br/detalhes.php?papel={query_tic}"
        r = session.get(url, timeout=15)
        r.raise_for_status()
        info = parse_fundamentus_page(r.text)
        # manter o ticker original pedido (para rastreabilidade)
        info['ticker'] = orig_tic
        rows.append(info)
        cols_union.update(info.keys())
        sleep(pause)

    # montar DataFrame (colunas ordenadas por união)
    cols = ['ticker'] + sorted([c for c in cols_union if c != 'ticker'])
    df = pd.DataFrame(rows, columns=cols)

    # ----- Excluir colunas solicitadas (oscilações, anos, etc.) -----
    drop_patterns = re.compile(r'(oscil|dia\b|mes\b|30\b|2020|2021|2022|2023|2024|2025)', re.I)
    df = df[[c for c in df.columns if not drop_patterns.search(c)]]

    # ----- Campo Empresa: capitalizar e manter siglas ON, PN, N1, N2, NM em maiúsculas -----
    if 'empresa' in df.columns:
        def format_empresa(nome):
            if not isinstance(nome, str) or nome.strip() == "":
                return nome
            partes = nome.split()
            out = []
            for p in partes:
                pu = p.upper()
                if pu in ['ON','PN','N1','N2','NM']:
                    out.append(pu)
                else:
                    out.append(p.capitalize())
            return " ".join(out)
        df['empresa'] = df['empresa'].apply(format_empresa)

    # ----- Conversão/format de números: valores > 1000 -> inteiros, sem exponencial -----
    for col in df.columns:
        ser = pd.to_numeric(df[col], errors='coerce')
        if ser.notna().any():
            # aplicar: se abs(x) >= 1000 transformar em int(round(x)) e armazenar como object para evitar notação
            def conv(x):
                if pd.isna(x):
                    return x
                try:
                    if abs(x) >= 1000:
                        return int(round(x))
                    # manter floats (indicadores) com 2 decimais quando não inteiros grandes
                    if float(x).is_integer():
                        return int(x)
                    return float(x)
                except:
                    return x
            df[col] = ser.apply(conv)
            df[col] = df[col].astype(object)
        else:
            df[col] = df[col].astype(object)

    # ----- Reordenar colunas conforme pedido (best-effort) -----
    desired_order = [
        "ticker","cotacao","data_ultima_cotacao","min_52_sem","max_52_sem","vol_med_2m",
        "setor","valor_mercado","valor_firma","nro_acoes","pl","lpa","p_vp","vpa","p_ebit",
        "marg_bruta","psr","marg_ebit","p_ativos","marg_liquida","p_cap_giro","p_ativ_circ_liq",
        "roic","dividend_yield","roe","ev_ebitda","liquidez_corr","ev_ebit","div_br_patrim",
        "cres_rec_5a","giro_ativos","ativo","disponibilidades","ativo_circulante","div_bruta",
        "div_liquida","patrimonio_liquido","receita_liquida_12m","ebit","lucro_liquido_12m"
    ]
    final_cols = []
    existing = list(df.columns)
    used = set()
    for want in desired_order:
        match = None
        if want in existing:
            match = want
        else:
            for c in existing:
                if c in used:
                    continue
                cl = c.lower()
                if want.replace('_',' ') in cl or all(tok in cl for tok in want.split('_') if tok):
                    match = c
                    break
        if match:
            final_cols.append(match)
            used.add(match)
    for c in existing:
        if c not in used:
            final_cols.append(c)
    fundamentus = df.reindex(columns=final_cols)

    # format display options to avoid scientific notation
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 400)
    pd.set_option('display.float_format', lambda x: '%.0f' % x if abs(x) >= 1000 else ('%.2f' % x))

    return fundamentus

# ---------------- Execução exemplo ----------------

if __name__ == "__main__":
    '''tickers = ['^BVSP','ABEV3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CMIN3.SA','CPFE3.SA','CPLE6.SA',
               'ELET6.SA','EMBR3.SA','ITUB4.SA','IVVB11.SA','JHSF3.SA','KLBN11.SA','MBRF3.SA','LEVE3.SA',
               'LREN3.SA','PETR4.SA','POMO4.SA','RECV3.SA','SBSP3.SA','TGMA3.SA','VALE3.SA','VIVA3.SA',
               'VIVT3.SA','WEGE3.SA']'''
    fundamentus = get_many_tickers_fundamentus_df(tickers)
    display(fundamentus)
    # print("\nDataFrame 'fundamentus' (uma linha por ticker):")
    # print(fundamentus.to_string(index=False))

,ticker,cotacao,data_ultima_cotacao,min_52_sem,max_52_sem,vol_med_2m,setor,valor_mercado,valor_firma,nro_acoes,pl,lpa,p_vp,vpa,p_ebit,marg_bruta,psr,marg_ebit,p_ativos,marg_liquida,p_cap_giro,p_ativ_circ_liq,roic,dividend_yield,roe,ev_ebitda,liquidez_corr,ev_ebit,div_br_patrim,cres_rec_5a,giro_ativos,ativo,disponibilidades,ativo_circulante,div_bruta,div_liquida,receita_liquida_12m,ebit,lucro_liquido_12m,12_meses,30_dias,cart_de_credito,depositos,ebit_ativo,empresa,papel,patrim_liq,rec_servicos,result_int_financ,subsetor,tipo,ult_balanco_processado,ultimos_12_meses
0,ABEV3,12.71,31/10/2025,10.52,14.29,358863000,Bebidas,200330000000,183384000000,15761600000,12.49,1.02,2.18,5.84,9.68,51.80,2.21,22.90,1.42,18.20,37.35,-32.91,20.20,8.30,17.40,6.69,1.15,8.86,0.03,5.90,0.64,140755000000,19840900000,41940200000,2894520000,-16946400000,20847300000,4892670000,4745130000,6.39,2.42,NaN,NaN,14.70,Ambev S/a ON,ABEV3,92027400000,NaN,NaN,Cervejas e Refrigerantes,ON,30/09/2025,Últimos 3 meses
1,BBAS3,21.90,31/10/2025,18.35,29.30,653646000,Intermediários Financeiros,125505000000,NaN,5730830000,4.57,4.79,0.72,30.49,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,7.90,15.70,NaN,NaN,NaN,NaN,7.10,NaN,2400620000000,NaN,NaN,NaN,NaN,NaN,NaN,3085410000,-11.99,-0.99,0.00,0.00,0.00,Banco Do Brasil S.a. ON,BBAS3,174734000000,4937150000,6359170000,Bancos,ON,30/06/2025,Últimos 3 meses
2,BBDC4,18.16,31/10/2025,10.58,18.83,507838000,Intermediários Financeiros,192351000000,NaN,10592000000,8.36,2.17,1.13,16.01,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,7.40,13.60,NaN,NaN,NaN,NaN,-6.10,NaN,1753560000000,NaN,NaN,NaN,NaN,NaN,NaN,6204590000,30.95,3.35,0.00,0.00,0.00,Banco Bradesco S.a. PN N1,BBDC4,169590000000,5846140000,7793600000,Bancos,PN N1,30/09/2025,Últimos 3 meses
3,BBSE3,32.85,31/10/2025,29.75,40.42,166730000,Previdência e Seguros,65700000000,NaN,2000000000,7.37,4.46,6.26,5.25,6.63,NaN,NaN,NaN,3.07,0.00,NaN,NaN,NaN,12.90,85.00,NaN,NaN,NaN,NaN,NaN,NaN,21376900000,0.00,0.00,0.00,0.00,0.00,2635520000,2415910000,6.94,0.58,NaN,NaN,46.30,Bbseguridade ON NM,BBSE3,10494000000,NaN,NaN,Seguradoras,ON NM,30/06/2025,Últimos 3 meses
4,CMIN3,6.07,31/10/2025,4.46,6.16,24790600,Mineração,33296000000,28196200000,5485340000,14.99,0.40,3.60,1.68,7.16,47.40,1.99,27.80,0.93,13.30,4.46,-3.41,23.70,4.60,24.00,4.82,1.78,6.06,1.00,-2.00,0.47,35991100000,14369600000,16982300000,9269770000,-5099840000,4038200000,952020000,115858000,4.79,9.96,NaN,NaN,12.90,Csnmineracao ON N2,CMIN3,9237280000,NaN,NaN,Minerais Metálicos,ON N2,30/06/2025,Últimos 3 meses
5,CPFE3,41.60,31/10/2025,28.79,41.68,51430700,Energia Elétrica,47933800000,72654900000,1152250000,8.87,4.69,2.23,18.64,4.25,31.50,1.09,25.60,0.60,13.00,-21.15,-1.18,15.80,6.70,25.20,5.46,0.88,6.45,1.35,5.10,0.56,79238200000,4210650000,16105800000,28931800000,24721100000,10549100000,2574690000,1124320000,36.83,6.29,NaN,NaN,14.20,Cpfl Energia S.a. ON,CPFE3,21481800000,NaN,NaN,Energia Elétrica,ON,30/06/2025,Últimos 3 meses
6,CPLE6,13.92,31/10/2025,8.43,13.92,133550000,Energia Elétrica,41520700000,58502900000,2982810000,13.64,1.02,1.62,8.58,9.75,21.80,1.74,17.80,0.68,12.70,28.94,-1.80,7.70,4.60,11.90,10.23,1.13,13.74,0.78,2.10,0.39,60742000000,2905740000,12093900000,19887900000,16982100000,6225150000,1046630000,572140000,49.55,7.99,NaN,NaN,7.00,Copel Pnb,CPLE6,25596100000,NaN,NaN,Energia Elétrica,PNB,30/06/2025,Últimos 3 meses
7,ELET6,59.39,31/10/2025,35.41,59.39,73351200,Energia Elétrica,137110000000,177562000000,2308630000,20.68,2.87,1.16,51.42,9.16,45.20,3.14,34.30,0.50,15.20,5.92,-1.29,6.20,6.50,5.60,9.23,1.86,11.86,0.59,8.90,0.16,274960000000,29837100000,49985200000,70289900000,40452700000,10199000000,3629960000,-1325110000,53.51,6.89,NaN,NaN,5.40,Eletrobrás Pnb,ELET6,118707000000,NaN,NaN,Energia Elétrica,PNB,30/06/2025,Últimos 3 meses
8,EMBR3,86.99,31/10/2025,48.42,89.37,398490000,Material de Transporte,64413100000,69584800000,740465000,30.13,2.89,3.54,24.57,14.60,18.60,1.62,11.10,0.98,5.30,7.34,-6.06,8.40,0.10,11.80,11.88,1.33,15.77,0.66,13.00,0.60,658

In [47]:
# Autentication in Google Docs (only once)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [48]:
# Open workbook and worksheets
wb = gc.open('Quotes')
wssfundamentus = wb.worksheet('Fundamentus')

In [49]:
# Write data in the worksheet
wssfundamentus.update([fundamentus.columns.values.tolist()] + fundamentus.fillna('').values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'Fundamentus!A1:BA26',
 'updatedRows': 26,
 'updatedColumns': 53,
 'updatedCells': 1378}